In [49]:
import glob
import os
import re
from collections import Counter

In [2]:
glob.glob('*.menu')

['2024-eclipse.menu',
 '2025-parker-instrument-checkout.menu',
 'cme-1074.menu',
 'daily-with-polarizations.menu',
 'daily.menu',
 'dark.menu',
 'engineering-diffuser-calibration.menu',
 'engineering-find-o1-focus.menu',
 'engineering-find_imageDrift.menu',
 'engineering-measure-distortion.menu',
 'engineering-measure-occulter.menu',
 'engineering-spectrograph validation.menu',
 'flat.menu',
 'parker-observations.menu',
 'poor-weather.menu',
 'waves-options.menu']

In [ ]:
for menu_file in glob.glob('*.menu'):
    with open(menu_file, 'r') as f:
        content = f.read()
        for line in content.splitlines():
            if ".cbk" in line:
                line = line.split('#')[0]
                if len(line) == 0:
                    continue
                file_path = line.split()[0]
                full_path = os.path.join("scripts", file_path)
                if not os.path.exists(full_path):
                    print(f"{menu_file} cookbook {file_path} does not exist {full_path})")
                with open(full_path, 'r') as cbk_file:
                    cbk_content = cbk_file.read()
                    if ".rcp" in cbk_content:
                        print(f"{menu_file} cookbook {file_path} missing RECIPE_USE")

In [ ]:
def check_recipe_name(recipe_path):
    if not  os.path.exists(recipe_path):
        print(f"Recipe {recipe_path} does not exist")
        return
    fName = recipe_path.split('\\')[-1].replace('.rcp','')
    parts = fName.split('_')
    filter = None
    waves = None
    beams = None
    step_size = None
    sums = None
    continuum = None
    reps = None
    if parts[0].isdigit():
        filter = parts[0]
       # parts = parts[1:]
    for part in parts[1:]:
        numeric = re.match(r'\d+(\.\d+)?', part)
        if numeric:
            value = float(numeric.group())
            if 'wave' in part:
                waves = value
            elif 'step' in part:
                step_size = value
            elif 'beam' in part:
                beams = int(value)
            elif 'sums' in part:
                sums = int(value)
            elif 'reps' in part:
                reps = int(value)
        if 'both'  in part.lower() or 'red'  in part.lower() or 'blue' in part.lower():
            continuum = part
    #    print(float(re.match(r'\d+', part).group()) i)
  #      if 'wave' in part:

   #         wave_part = part
   #         break
   # filter = fName.split('_')[0]
   # wave = fName.split('_')[1]
    #s
    return {"filter": filter, "waves": waves, "step_size": step_size, "beams": beams, "sums": sums, "reps": reps, "continuum": continuum}
    #print(f"{fName} filter {filter} Waves: {waves}, Step Size: {step_size}, Beams: {beams}, Sums: {sums}, Reps: {reps} continuum: {continuum}")

def verify_recipe_contents(recipe_path):
    if not  os.path.exists(recipe_path):
        print(f"Recipe {recipe_path} does not exist")
        return
    cam = []
    cont = []
    wave = []
    sums = []
    name_info = check_recipe_name(recipe_path)
    with open(recipe_path, 'r') as f:
        content = f.read()
        for line in content.splitlines():
            line = line.lower()
            if line.startswith("data") and len(line.split("#")[0].split()) ==5:
                parts = line.split()
                cam.append(parts[1])
                cont.append(parts[2])
                wave.append(parts[3])
                sums.append(parts[4])
    if len(wave) == 0:
       # print(f"Recipe {recipe_path} has no data lines")
        return
    if len(set(cont)) > 1:
        print(f"Recipe {recipe_path} has multiple continuum settings: {set(cont)}")
    elif name_info["continuum"] is None:
        print(f"Recipe {recipe_path} continuum not specified in name but found {cont[0]} in recipe")
    elif name_info["continuum"].lower() != cont[0].lower():
        print(f"Recipe {recipe_path} continuum in name {name_info['continuum']} does not match recipe content {cont[0]}")
   # wave_counter = Counter(wave)
    if len(set(Counter(wave).values())) > 1:
        print(f"Recipe {recipe_path} has inconsistent waves per")
    elif name_info["waves"] != len(set(wave)):
        print(f"Recipe {recipe_path} waves in name {name_info['waves']} does not match recipe content {len(set(wave))}")
    if len(set(Counter(sums).values())) > 1:
        print(f"Recipe {recipe_path} has inconsistent sums per")
    elif name_info["sums"] != int(sums[0]):
        print(f"Recipe {recipe_path} sums in name {name_info['sums']} does not match recipe content {sums[0]}")
    #if name_info["continuum"] is not None:
     #   if len(set(cont)) == 1:
      #      if name_info["continuum"].lower() != set(cont).pop():
       #         print(f"Recipe {recipe_path} continuum in name {name_info['continuum']} does not match recipe content {set(cont)}")
    #print(f"Recipe {recipe_path} has cameras: {len(set(cam))}, continuum: {set(cont)}, waves: {len(set(wave))}, sums: {set(sums)}")   

#verify_recipe_contents(rcp_path)
for recipe_file in glob.glob('scripts/*.rcp'):
    verify_recipe_contents(recipe_file)

Recipe scripts\1074_07drift_2beam_16sums_1rep_both.rcp has no data lines
Recipe scripts\1074_1_2beams_4sums.rcp continuum not specified in name but found both in recipe
Recipe scripts\1074_1_2beams_4sums.rcp waves in name None does not match recipe content 1
Recipe scripts\1074_40wave_0.12step_2beam_16sums_1reps_BOTH.rcp has inconsistent waves per
Recipe scripts\1074_focus.rcp has no data lines
Recipe scripts\1074_FW.rcp has no data lines
Recipe scripts\1074_Pol_Calibrate.rcp has no data lines
Recipe scripts\1078_07wave_0.06step_2beam_16sums_1reps_BOTH.rcp has inconsistent waves per
Recipe scripts\1079_07wave_0.06step_2beam_16sums_1reps_BOTH.rcp has inconsistent waves per
Recipe scripts\1079_07wave_0.06step_2beam_16sums_4reps_BOTH.rcp waves in name 7.0 does not match recipe content 8
Recipe scripts\1079_focus.rcp has no data lines
Recipe scripts\1079_FW.rcp has no data lines
Recipe scripts\1079_Pol_Calibrate.rcp has no data lines
Recipe scripts\1083_focus-front.rcp continuum not specif

In [22]:
rcp_path = "scripts/637_07wave_0.03step_2beam_16sums_4reps_BOTH.rcp"

In [ ]:
re.match(r'\d+', s)